In [120]:
import tensorflow as tf
import numpy as np

In [121]:
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()
mnist_images, mnist_labels = mnist_images[:1000], mnist_labels[:1000]
dataset = tf.data.Dataset.from_tensor_slices((tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32), tf.cast(mnist_labels,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [122]:
import onnx
from onnx_tf.backend import prepare
loaded_model = onnx.load("onnx_model/tf_model.onnx")
tf_loaded_model = prepare(loaded_model)

In [123]:
total_test_accuracy = []
total_test_loss = []

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def predict_test(images, labels):
  # Reset the metrics at the start of the next epoch
  test_loss.reset_states()
  test_accuracy.reset_states()
  true_labels = []
  predicted_labels = []
  for i, item in enumerate(images):
      prediction = tf_loaded_model.run(item, training=False)
      pred = np.array(prediction).squeeze()
      true_labels.append([labels[i].numpy()])
      predicted_labels.append(pred)
  t_loss = loss_object(true_labels, predicted_labels)
  return test_loss(t_loss), test_accuracy(labels, predicted_labels)

In [124]:
for test_images, test_labels in dataset:
  ls, acc = predict_test(test_images, test_labels)
  loss = test_loss.result()
  accuracy = test_accuracy.result() * 100
  total_test_accuracy.append(accuracy.numpy())
  total_test_loss.append(loss.numpy())

print("Mean test loss: {}".format(np.mean(total_test_loss)))

Mean test loss: 2.295968532562256


In [125]:
import tf2onnx

if not os.path.exists('tf_model'):
    os.makedirs('tf_model')

if not os.path.exists('onnx_local_model'):
    os.makedirs('onnx_local_model')

tf.saved_model.save(mnist_model, 'tf_model')

NameError: name 'mnist_model' is not defined

In [ ]:
%%bash

python -m tf2onnx.convert --saved-model tf_model --output onnx_local_model/mnist_model.onnx --opset 7